In [4]:
phoneme_dict = {
    'er': 0,
    'aa': 1,
    'ahn': 2,
    'ah': 3,
    'tq': 4,
    'nx': 5,
    'jh': 6,
    'own': 7,
    'ihn': 8,
    'hh': 9,
    'ey': 10,
    'v': 11,
    'l': 12,
    'dh': 13,
    'uh': 14,
    'f': 15,
    'r': 16,
    'uw': 17,
    'aen': 18,
    's': 19,
    'ao': 20,
    'b': 21,
    't': 22,
    'ih': 23,
    'th': 24,
    'oy': 25,
    'SIL': 26,
    'ch': 27,
    'eng': 28,
    'eh': 29,
    'p': 30,
    'sh': 31,
    'ehn': 32,
    'ay': 33,
    'z': 34,
    'w': 35,
    'g': 36,
    'm': 37,
    'aw': 38,
    'em': 39,
    'el': 40,
    'ae': 41,
    'd': 42,
    'k': 43,
    'dx': 44,
    'en': 45,
    'y': 46,
    'ow': 47,
    'ng': 48,
    'iy': 49,
    'zh': 50,
    'n': 51
}

excluded_segments = {"aan","iyn","ayn","eyn","awn","aon","oyn","uwn","uhn","a","h","e","q","ern","uw ix","i","an","hhn","no","j","x", "id", "ah ix", "ih l", "ah n", "ah r", "ah l", "VOCNOISE", "NOISE", "EXCLUDE", "<EXCLUDE>",
                     "IVER y", "IVER-LAUGH", None, ", ", "None", "{E_TRANS}", "<exclude-Name>", "LAUGH", "IVER",
                     "{B_TRANS}", "UNKNOWN", "<EXCLUDE-name>"}

In [4]:
import librosa
import buckeye
import cleaneddata
import numpy as np
print("fetching the buckeye dataset .....")

phoneme_intervals = []
phoneme_labels = []
frame_duration = 0.01
excluded_segments = cleaneddata.excluded_segments
phoneme_dict = cleaneddata.phoneme_dict
sp = 0
corpus = buckeye.corpus('D:\\buckeye_dataset', load_wavs=True)

for speaker in corpus:
    if sp == 2:
        break
    for track in speaker:
        audio_file = fr"E:\buckeye_wav\{track.name}.wav"
        audio, sr = librosa.load(audio_file)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, hop_length=441)

        for phone in track.phones:
            if phone.dur > 0.01 and phone.seg not in excluded_segments:
                start_time = phone.beg
                end_time = phone.end
                phoneme_intervals.append((start_time, end_time))
                phoneme_labels.append(phoneme_dict[phone.seg])
    sp += 1

print("sad2")

frame_indices = []
for interval in phoneme_intervals:
    start_frame = int(interval[0] / frame_duration)
    end_frame = int(interval[1] / frame_duration)
    frame_indices.append((start_frame, end_frame))

print("sad3")

aligned_frames = []
aligned_labels = []
for indices in frame_indices:
    start_frame, end_frame = indices
    frames = mfccs[:, start_frame:end_frame]
    aligned_frames.append(frames)
    aligned_labels.append(phoneme_labels[len(aligned_frames) - 1])

# Print the length and shape of each list
print("Length of phoneme_intervals:", len(phoneme_intervals))
print("Length of phoneme_labels:", len(phoneme_labels))
print("Length of frame_indices:", len(frame_indices))
print("Length of aligned_frames:", len(aligned_frames))
print("Length of aligned_labels:", len(aligned_labels))

print("Shape of mfccs:", mfccs.shape)

# Determine the maximum shape of the subarrays


# Calculate the average length of the aligned frames

scaling_factor = 6


# Calculate the average length of the aligned frames
average_length = int(np.mean([len(frames[0]) for frames in aligned_frames]) * scaling_factor)

# Truncate longer sequences and pad shorter sequences
truncated_aligned_frames = []
for frames in aligned_frames:
    if len(frames[0]) > average_length:
        truncated_frames = frames[:, :average_length]
    else:
        padding = np.zeros((frames.shape[0], average_length - len(frames[0])))
        truncated_frames = np.concatenate((frames, padding), axis=1)
    truncated_aligned_frames.append(truncated_frames)

# Convert the list to a NumPy array
aligned_frames = np.array(truncated_aligned_frames)


# Print the length and shape of the modified array
print("Length of truncated_aligned_frames:", len(truncated_aligned_frames))
print("Shape of aligned_frames:", aligned_frames.shape)



# Print the length and shape of the modified array
aligned_labels = np.array(aligned_labels)
print("Shape of aligned_labels:", aligned_labels.shape)


print("bye")

fetching the buckeye dataset .....
sad2
sad3
Length of phoneme_intervals: 36843
Length of phoneme_labels: 36843
Length of frame_indices: 36843
Length of aligned_frames: 36843
Length of aligned_labels: 36843
Shape of mfccs: (20, 21642)
Length of truncated_aligned_frames: 36843
Shape of aligned_frames: (36843, 20, 24)
Shape of aligned_labels: (36843,)
bye


In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))



[]


In [9]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Assuming alignes
# d_frames is X and aligned_labels is y
#print(tf.config.list_physical_devices('GPU'))
X = aligned_frames  # Shape: (num_samples, timesteps, input_dim)
Y = aligned_labels  # Shape: (num_samples,)



# Assuming aligned_frames is your input data X

# Reshape the input data to 2D (flatten the frames)


# Now X_normalized contains the normalized input data

#num_classes = len(np.unique(y))  # Number of unique phoneme classes
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=12)

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(20, 24), return_sequences=True))  # Input shape: (timesteps, input_dim)
model.add(Dropout(0.2))  # Dropout layer with 10% dropout rate
model.add(LSTM(128))  # Hidden LSTM layer
model.add(Dropout(0.2))  # Dropout layer with 10% dropout rate
model.add(Dense(52, activation='sigmoid'))  # Output shape: (num_classes,)


early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_split=0., callbacks=[early_stopping])

# Make predictions
predictions = model.predict(X)

# Evaluate the model
accuracy = model.evaluate(X_test, Y_test)
print("Accuracy:", accuracy)

Epoch 1/50
